# Quick Analysis

In [1]:
from pathlib import Path  # Python Standard Library
import pandas as pd  # pip install pandas openpyxl
from pyecharts import options as opts  # pip install pyecharts
from pyecharts.charts import Bar, Calendar, Tab

In [2]:
# Read Excel file (Financial_Data.xlsx) into a pandas DataFrame.
df = pd.read_excel(
    io= Path.cwd() / "Financial_Data.xlsx",
    engine= "openpyxl",
    sheet_name= "Orders",
    skiprows= 2,
    usecols= "B:T",
    nrows= 3312,
)
df.head()

,Order ID,Order Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,CA-2017-114412,2021-04-15,Standard Class,AA-10480,Andrew Allen,Consumer,United States,Concord,North Carolina,28027,South,OFF-PA-10002365,Office Supplies,Paper,Xerox 1967,15.552,3,0.2,5.4432
1,US-2017-156909,2021-07-16,Second Class,SF-20065,Sandra Flanagan,Consumer,United States,Philadelphia,Pennsylvania,19140,East,FUR-CH-10002774,Furniture,Chairs,"Global Deluxe Stacking Chair, Gray",71.372,2,0.3,-1.0196
2,CA-2017-107727,2021-10-19,Second Class,MA-17560,Matt Abelman,Home Office,United States,Houston,Texas,77095,Central,OFF-PA-10000249,Office Supplies,Paper,Easy-staple paper,29.472,3,0.2,9.9468
3,CA-2017-120999,2021-09-10,Standard Class,LC-16930,Linda Cazamias,Corporate,United States,Naperville,Illinois,60540,Central,TEC-PH-10004093,Technology,Phones,Panasonic Kx-TS550,147.168,4,0.2,16.5564
4,CA-2017-139619,2021-09-19,Standard Class,ES-14080,Erin Smith,Corporate,United States,Melbourne,Florida,32935,South,OFF-ST-10003282,Office Supplies,Storage,"Advantus 10-Drawer Portable Organizer, Chrome ...",95.616,2,0.2,9.5616


In [3]:
# Get a quick info about our dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3312 entries, 0 to 3311
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Order ID       3312 non-null   object        
 1   Order Date     3312 non-null   datetime64[ns]
 2   Ship Mode      3312 non-null   object        
 3   Customer ID    3312 non-null   object        
 4   Customer Name  3312 non-null   object        
 5   Segment        3312 non-null   object        
 6   Country        3312 non-null   object        
 7   City           3312 non-null   object        
 8   State          3312 non-null   object        
 9   Postal Code    3312 non-null   int64         
 10  Region         3312 non-null   object        
 11  Product ID     3312 non-null   object        
 12  Category       3312 non-null   object        
 13  Sub-Category   3312 non-null   object        
 14  Product Name   3312 non-null   object        
 15  Sales          3312 n

In [4]:
# Add month column to our dataframe
df['Month'] = df['Order Date'].dt.month
df.head(3)

,Order ID,Order Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Month
0,CA-2017-114412,2021-04-15,Standard Class,AA-10480,Andrew Allen,Consumer,United States,Concord,North Carolina,28027,South,OFF-PA-10002365,Office Supplies,Paper,Xerox 1967,15.552,3,0.2,5.4432,4
1,US-2017-156909,2021-07-16,Second Class,SF-20065,Sandra Flanagan,Consumer,United States,Philadelphia,Pennsylvania,19140,East,FUR-CH-10002774,Furniture,Chairs,"Global Deluxe Stacking Chair, Gray",71.372,2,0.3,-1.0196,7
2,CA-2017-107727,2021-10-19,Second Class,MA-17560,Matt Abelman,Home Office,United States,Houston,Texas,77095,Central,OFF-PA-10000249,Office Supplies,Paper,Easy-staple paper,29.472,3,0.2,9.9468,10


In [5]:
# group dataframe by months
grouped_by_months = df.groupby(by=['Month']).sum()[['Sales', 'Profit']]
grouped_by_months.head(3)

,Sales,Profit
Month,,
1,43971.3740,7140.4391
2,20301.1334,1613.8720
3,58872.3528,14751.8915


In [6]:
# plot sales & profit by months
bar_chart_by_month = (
    Bar()
    .add_xaxis(grouped_by_months.index.tolist())
    .add_yaxis('Sales', grouped_by_months['Sales'].round(0).tolist())
    .add_yaxis('Profit', grouped_by_months['Profit'].round(0).tolist())
    .set_global_opts(
        title_opts=opts.TitleOpts(title='Sales & Profit by month', subtitle='in USD')
    )
)

In [7]:
bar_chart_by_month.render_notebook()

In [8]:
# group data by sub-category
grouped_by_sub_category = df.groupby(by='Sub-Category', as_index=False).sum().sort_values(by=['Sales'])
grouped_by_sub_category.head(3)

,Sub-Category,Postal Code,Sales,Quantity,Discount,Profit,Month
8,Fasteners,3811489,857.594,249,4.2,304.9489,481
7,Envelopes,3714485,3378.574,241,6.2,1441.7590,541
10,Labels,5948688,3861.216,448,7.2,1744.6093,830


In [9]:
# plot sales & profit by subcategroy
bar_chart_by_subcategory = (
    Bar()
    .add_xaxis(grouped_by_sub_category['Sub-Category'].tolist())
    .add_yaxis('Sales', grouped_by_sub_category['Sales'].round(0).tolist())
    .add_yaxis('Profit', grouped_by_sub_category['Profit'].round(0).tolist())
    .reversal_axis()
    .set_series_opts(label_opts=opts.LabelOpts(position='right'))
    .set_global_opts(
        title_opts=opts.TitleOpts(title='Sales & Profit by subcategory', subtitle='in USD')
    )
)

In [10]:
bar_chart_by_subcategory.render_notebook()

In [11]:
# Plot sales calendar
sales_by_date = df.groupby(by='Order Date').sum()[['Sales']].round(0)
sales_by_date = sales_by_date.reset_index()
sales_by_date.head(3)

,Order Date,Sales
0,2021-01-01,1482.0
1,2021-01-02,2080.0
2,2021-01-03,2070.0


In [12]:
data = sales_by_date[['Order Date', 'Sales']].values.tolist()
data

[[Timestamp('2021-01-01 00:00:00'), 1482.0],
 [Timestamp('2021-01-02 00:00:00'), 2080.0],
 [Timestamp('2021-01-03 00:00:00'), 2070.0],
 [Timestamp('2021-01-06 00:00:00'), 34.0],
 [Timestamp('2021-01-07 00:00:00'), 3396.0],
 [Timestamp('2021-01-08 00:00:00'), 893.0],
 [Timestamp('2021-01-09 00:00:00'), 274.0],
 [Timestamp('2021-01-12 00:00:00'), 849.0],
 [Timestamp('2021-01-13 00:00:00'), 4619.0],
 [Timestamp('2021-01-14 00:00:00'), 1056.0],
 [Timestamp('2021-01-15 00:00:00'), 2140.0],
 [Timestamp('2021-01-16 00:00:00'), 6230.0],
 [Timestamp('2021-01-19 00:00:00'), 2014.0],
 [Timestamp('2021-01-20 00:00:00'), 768.0],
 [Timestamp('2021-01-21 00:00:00'), 2547.0],
 [Timestamp('2021-01-22 00:00:00'), 4212.0],
 [Timestamp('2021-01-23 00:00:00'), 450.0],
 [Timestamp('2021-01-24 00:00:00'), 419.0],
 [Timestamp('2021-01-26 00:00:00'), 3055.0],
 [Timestamp('2021-01-27 00:00:00'), 332.0],
 [Timestamp('2021-01-28 00:00:00'), 1008.0],
 [Timestamp('2021-01-29 00:00:00'), 295.0],
 [Timestamp('2021-01

In [13]:
# calculate min / max sales
max_sales = df['Sales'].max()
min_sales = df['Sales'].min()

In [14]:
sales_calendar = (
    Calendar()
    .add('', data, calendar_opts=opts.CalendarOpts(range_='2021'))
    .set_global_opts(
        title_opts=opts.TitleOpts(title='Sales Calendar', subtitle='in USD'),
        legend_opts=opts.LegendOpts(is_show=False),
        visualmap_opts=opts.VisualMapOpts(
                max_=max_sales,
                min_=min_sales,
                orient='horizontal',
                is_piecewise=False,
                pos_top='230px',
                pos_left='100px',
        )
    )
)

In [15]:
sales_calendar.render_notebook()

In [16]:
# Create HTML file to combine all charts
tab = Tab(page_title='Sales & Profit Overview')
tab.add(bar_chart_by_month, 'Sales & Profit by months')
tab.add(bar_chart_by_subcategory, 'Sales & Profit by subcategory')
tab.add(sales_calendar, 'Sales Calendar')
tab.render(Path.cwd() / 'Sales & Profit Overview.html')

'C:\\Users\\Bananatree\\Documents\\YT_Tutorials\\01_Projects\\84_Interactive_HTML_Dashboard\\YOUTUBE_UPLOAD_FILES\\Sales & Profit Overview.html'